# Lab 12-1 many to one
### word sentiment classification 
* many to one 
* variable input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.nn import embedding_lookup
from tensorflow.nn import dynamic_rnn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.12.0


### Prepairing dataset

In [2]:
# example data
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]

# generating a token dictionary
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)

['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
# converting token to Idx
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding 
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence, padding='post',
                       truncating='post')

print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


### Creating model

In [5]:
class Model:
    def __init__(self, x, x_len, y, hidden_dim, num_classes, dic):
        # input
        self.x = x
        self.x_len = x_len
        self.y = y
        self.dic = dic
            
        # token representation
        one_hot = tf.Variable(np.eye(len(self.dic)), dtype=tf.float32, trainable=False)
        x_batch = embedding_lookup(params=one_hot, ids=self.x)
            
        # rnn
        rnn_cell = layers.SimpleRNNCell(units=hidden_dim)
        _, state = dynamic_rnn(cell=rnn_cell, inputs=x_batch, sequence_length=self.x_len,
                               dtype=tf.float32)
        
        # output
        self.score = layers.Dense(units=num_classes)(state)

        # loss
        self.ce_loss = tf.losses.sparse_softmax_cross_entropy(labels=self.y, logits=self.score)
            
        # prediction
        self.prediction = tf.argmax(input=self.score, axis=-1)
            
    def predict(self, sess, x, x_len):
        feed_prediction = {self.x : x, self.x_len : x_len}
        return sess.run(self.prediction, feed_dict=feed_prediction)

### Training model

In [6]:
# training hyper-paramter
lr = .01
epochs = 30
batch_size = 2

# generating data pipeline 
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data, x_data_len))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size=2)

print(tr_dataset)
tr_iterator = tr_dataset.make_initializable_iterator()
x_mb, y_mb, x_mb_len = tr_iterator.get_next()

# creating model
model = Model(x=x_mb, y=y_mb, x_len=x_mb_len, hidden_dim=10, num_classes=2, dic=char2idx)

<BatchDataset shapes: ((?, 10), (?,), (?,)), types: (tf.int32, tf.int32, tf.int32)>


In [7]:
# creating optimizer
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss=model.ce_loss)

In [8]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    # training
    sess.run(tr_iterator.initializer)
    try:
        while True:
            _, tr_loss = sess.run([training_op, model.ce_loss])
            avg_tr_loss += tr_loss
            tr_step += 1
    except tf.errors.OutOfRangeError:
        avg_tr_loss /= tr_step
        
    if (epoch + 1) % 5 == 0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch+1, avg_tr_loss))

epoch :   5, tr_loss : 0.404
epoch :  10, tr_loss : 0.129
epoch :  15, tr_loss : 0.038
epoch :  20, tr_loss : 0.015
epoch :  25, tr_loss : 0.009
epoch :  30, tr_loss : 0.006


### Checking performance

In [9]:
yhat = model.predict(sess, x_data, x_data_len)

In [10]:
print('acc : {:.2%}'.format(np.mean(yhat == y_data)))

acc : 100.00%
